In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import cv2

In [3]:
# Step 2: Load and Preprocess Dataset
data_dir = '/content/drive/MyDrive/bangla_digits'
digit_folders = [f'digit_{i}' for i in range(10)]
images = []
labels = []

In [10]:
for digit, folder in enumerate(digit_folders):
    folder_path = os.path.join(data_dir, folder)
    if not os.path.exists(folder_path):
        print(f"Error: Folder {folder_path} not found")
        raise FileNotFoundError(f"Folder {folder_path} not found")

    for i in range(1, 21):  # Images bangla_X_1.png to bangla_X_20.png
        img_name = f'bangla_{digit}_{i}.png'
        img_path = os.path.join(folder_path, img_name)
        if not os.path.exists(img_path):
            print(f"Warning: Image {img_path} not found, skipping")
            continue

        # Load image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Failed to load {img_path}, skipping")
            continue

        # Resize to 28x28 (like MNIST)
        img = cv2.resize(img, (28, 28))
        # Normalize pixel values to [0, 1]
        img = img.astype('float32') / 255.0
        # Reshape to (28, 28, 1) for CNN input
        img = np.expand_dims(img, axis=-1)

        images.append(img)
        labels.append(digit)


In [ ]:
X = np.array(images)
y = np.array(labels)

In [ ]:
print(f"Loaded {len(X)} images with shape {X.shape}")
print("Label distribution:")
print(pd.Series(y).value_counts())

Loaded 200 images with shape (200, 28, 28, 1)
Label distribution:
0    20
1    20
2    20
3    20
4    20
5    20
6    20
7    20
8    20
9    20
Name: count, dtype: int64


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
accuracies = []
classification_reports = []

for train_idx, test_idx in skf.split(X, y):
    print(f"\nFold {fold}")
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Split train into train+validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
    )

    print(f"Train samples: {len(X_train)}, Validation samples: {len(X_val)}, Test samples: {len(X_test)}")
    print("Test label distribution:")
    print(pd.Series(y_test).value_counts())

    # Step 5: Build CNN Model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')  # 10 classes (digits 0-9)
    ])

    # Compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Step 6: Train Model with Data Augmentation
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    # Fit model with augmentation
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Step 7: Evaluate Model
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"Fold {fold} Test Accuracy: {accuracy:.4f}")

    # Classification report
    report = classification_report(y_test, y_pred, target_names=[f'০ ({i})' for i in range(10)], output_dict=True)
    classification_reports.append(report)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=[f'০ ({i})' for i in range(10)]))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[f'০ ({i})' for i in range(10)], yticklabels=[f'০ ({i})' for i in range(10)])
    plt.title(f'Confusion Matrix - Fold {fold}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'/content/drive/MyDrive/bangla_digits/confusion_matrix_fold_{fold}.png')
    plt.close()

    # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'/content/drive/MyDrive/bangla_digits/training_plots_fold_{fold}.png')
    plt.close()

    fold += 1


Fold 1
Train samples: 144, Validation samples: 16, Test samples: 40
Test label distribution:
0    4
1    4
2    4
3    4
4    4
5    4
6    4
7    4
8    4
9    4
Name: count, dtype: int64
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.0590 - loss: 2.3649 - val_accuracy: 0.0625 - val_loss: 2.2978 - learning_rate: 0.0010
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.0619 - loss: 2.3213 - val_accuracy: 0.2500 - val_loss: 2.3008 - learning_rate: 0.0010
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.0661 - loss: 2.3114 - val_accuracy: 0.1250 - val_loss: 2.3036 - learning_rate: 0.0010
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.1409 - loss: 2.2962 - val_accuracy: 0.1250 - val_loss: 2.3054 - learning_rate: 0.0010
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0579 - loss: 2.2994 - val_accuracy: 0.0625 - val_loss: 2.3061 - learning_rate: 2.0000e-04
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.0780 - loss: 2.3033 - val_accuracy: 0.0625 - val_loss: 2.3070 - learning_rate: 2.0000e-04
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Fold 1 Test Accuracy: 0.0250

Classification Report:
              pr

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


Fold 2
Train samples: 144, Validation samples: 16, Test samples: 40
Test label distribution:
0    4
1    4
2    4
3    4
4    4
5    4
6    4
7    4
8    4
9    4
Name: count, dtype: int64
Epoch 1/50


...

/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 2534 (\N{BENGALI DIGIT ZERO}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Matplotlib currently does not support Bengali natively.
  fig.canvas.draw()
<ipython-input-10-a867b7bbb172>:67: UserWarning: Glyph 2534 (\N{BENGALI DIGIT ZERO}) missing from font(s) DejaVu Sans.
  plt.savefig(f'/content/drive/MyDrive/bangla_digits/confusion_matrix_fold_{fold}.png')
<ipython-input-10-a867b7bbb172>:67: UserWarning: Matplotlib currently does not support Bengali natively.
  plt.savefig(f'/content/drive/MyDrive/bangla_digits/confusion_matrix_fold_{fold}.png')



Fold 5
Train samples: 144, Validation samples: 16, Test samples: 40
Test label distribution:
0    4
1    4
2    4
3    4
4    4
5    4
6    4
7    4
8    4
9    4
Name: count, dtype: int64
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.1297 - loss: 2.3239 - val_accuracy: 0.0625 - val_loss: 2.3200 - learning_rate: 0.0010
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.1072 - loss: 2.2951 - val_accuracy: 0.0000e+00 - val_loss: 2.3224 - learning_rate: 0.0010
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.1039 - loss: 2.3073 - val_accuracy: 0.0625 - val_loss: 2.3129 - learning_rate: 0.0010
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.1700 - loss: 2.2990 - val_accuracy: 0.1875 - val_loss: 2.3086 - learning_rate: 0.0010
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1013 - loss: 2.2933 - val_accuracy: 0.1250 - val_loss: 2.3039 - learning_rate: 0.0010
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1143 - loss: 2.2913 - val_accuracy: 0.0625 - val_loss: 2.3026 - learning_rate: 0.0010
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.1172 - loss: 2.2851 - val_accuracy: 0.1250 - val_l

/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 2534 (\N{BENGALI DIGIT ZERO}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Matplotlib currently does not support Bengali natively.
  fig.canvas.draw()
<ipython-input-10-a867b7bbb172>:67: UserWarning: Glyph 2534 (\N{BENGALI DIGIT ZERO}) missing from font(s) DejaVu Sans.
  plt.savefig(f'/content/drive/MyDrive/bangla_digits/confusion_matrix_fold_{fold}.png')
<ipython-input-10-a867b7bbb172>:67: UserWarning: Matplotlib currently does not support Bengali natively.
  plt.savefig(f'/content/drive/MyDrive/bangla_digits/confusion_matrix_fold_{fold}.png')


In [ ]:
print("\nCross-Validation Summary:")
print(f"Mean Accuracy: {np.mean(accuracies):.4f} (+/- {np.std(accuracies):.4f})")

if classification_reports:
    avg_report = {f'০ ({i})': {'precision': 0, 'recall': 0, 'f1-score': 0} for i in range(10)}
    for report in classification_reports:
        for label in avg_report:
            for metric in ['precision', 'recall', 'f1-score']:
                avg_report[label][metric] += report[label][metric] / len(classification_reports)
    print("\nAverage Classification Report:")
    for label in avg_report:
        print(f"{label}:")
        print(f"  Precision: {avg_report[label]['precision']:.4f}")
        print(f"  Recall: {avg_report[label]['recall']:.4f}")
        print(f"  F1-score: {avg_report[label]['f1-score']:.4f}")


Cross-Validation Summary:
Mean Accuracy: 0.4300 (+/- 0.2164)

Average Classification Report:
০ (0):
  Precision: 0.3000
  Recall: 0.3500
  F1-score: 0.3033
০ (1):
  Precision: 0.2967
  Recall: 0.3000
  F1-score: 0.2856
০ (2):
  Precision: 0.5889
  Recall: 0.6500
  F1-score: 0.5945
০ (3):
  Precision: 0.3080
  Recall: 0.1500
  F1-score: 0.1605
০ (4):
  Precision: 0.6833
  Recall: 0.3500
  F1-score: 0.4243
০ (5):
  Precision: 0.2533
  Recall: 0.2500
  F1-score: 0.2476
০ (6):
  Precision: 0.4000
  Recall: 0.4000
  F1-score: 0.3889
০ (7):
  Precision: 0.5667
  Recall: 0.7000
  F1-score: 0.6076
০ (8):
  Precision: 0.4689
  Recall: 0.7500
  F1-score: 0.5675
০ (9):
  Precision: 0.4357
  Recall: 0.4000
  F1-score: 0.3924


In [ ]:
os.makedirs('/content/drive/MyDrive/bangla_digits', exist_ok=True)
try:
    model.save('/content/drive/MyDrive/bangla_digits/bangla_digit_model.keras')
    print("\nModel saved successfully")
except Exception as e:
    print(f"Error saving model: {e}")
    raise


Model saved successfully


In [ ]:
print("\nPredicting on sample images:")
sample_indices = np.random.choice(len(X_test), 5, replace=False)
for idx in sample_indices:
    img = X_test[idx]
    true_label = y_test[idx]
    pred = np.argmax(model.predict(np.expand_dims(img, axis=0)), axis=1)[0]

    plt.figure(figsize=(3, 3))
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(f'True: ০ ({true_label}), Pred: ০ ({pred})')
    plt.axis('off')
    plt.savefig(f'/content/drive/MyDrive/bangla_digits/sample_pred_{idx}.png')
    plt.close()
    print(f"Sample {idx}: True Label: ০ ({true_label}), Predicted: ০ ({pred})")


Predicting on sample images:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Sample 25: True Label: ০ (6), Predicted: ০ (9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


<ipython-input-13-8963581d931d>:12: UserWarning: Glyph 2534 (\N{BENGALI DIGIT ZERO}) missing from font(s) DejaVu Sans.
  plt.savefig(f'/content/drive/MyDrive/bangla_digits/sample_pred_{idx}.png')
<ipython-input-13-8963581d931d>:12: UserWarning: Matplotlib currently does not support Bengali natively.
  plt.savefig(f'/content/drive/MyDrive/bangla_digits/sample_pred_{idx}.png')


Sample 15: True Label: ০ (3), Predicted: ০ (0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Sample 20: True Label: ০ (5), Predicted: ০ (6)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Sample 18: True Label: ০ (4), Predicted: ০ (4)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Sample 35: True Label: ০ (8), Predicted: ০ (8)
